# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
#import "clean cities csv"
file = "../output_data/clean_cities_data.csv"
weather_df = pd.read_csv(file)
weather_df

,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
0,Mataura,-46.19,168.86,282.98,49,0,0.15,2020-07-19 01:35:41
1,Jamestown,42.10,-79.24,296.86,69,1,2.72,2020-07-19 01:35:41
2,Lüderitz,-26.65,15.16,291.15,25,0,2.10,2020-07-19 01:35:41
3,Ushuaia,-54.80,-68.30,273.15,94,75,4.10,2020-07-19 01:35:41
4,Rikitea,-23.12,-134.97,294.16,82,100,5.00,2020-07-19 01:35:41
...,...,...,...,...,...,...,...,...
560,Vangaindrano,-23.35,47.60,291.08,84,53,2.27,2020-07-19 01:37:06
561,Jonesboro,35.84,-90.70,303.81,52,1,4.10,2020-07-19 01:33:40
562,Vila Franca do Campo,37.72,-25.43,293.15,82,20,4.10,2020-07-19 01:37:06
563,Liaocheng,36.44,115.96,300.56,56,96,5.60,2020-07-19 01:37:07


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#access maps with unique API key
gmaps.configure(api_key = g_key)

In [24]:
#create coordinates
records = weather_df[["Latitude", "Longitude"]].to_records(index=False)
coordinates = list(records)
weights = weather_df['Humidity']
#customize map
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights = weights)

#assign marker layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#drop null values
vacation_df = weather_df.dropna(axis=1)
vacation_df

,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
0,Mataura,-46.19,168.86,282.98,49,0,0.15,2020-07-19 01:35:41
1,Jamestown,42.10,-79.24,296.86,69,1,2.72,2020-07-19 01:35:41
2,Lüderitz,-26.65,15.16,291.15,25,0,2.10,2020-07-19 01:35:41
3,Ushuaia,-54.80,-68.30,273.15,94,75,4.10,2020-07-19 01:35:41
4,Rikitea,-23.12,-134.97,294.16,82,100,5.00,2020-07-19 01:35:41
...,...,...,...,...,...,...,...,...
560,Vangaindrano,-23.35,47.60,291.08,84,53,2.27,2020-07-19 01:37:06
561,Jonesboro,35.84,-90.70,303.81,52,1,4.10,2020-07-19 01:33:40
562,Vila Franca do Campo,37.72,-25.43,293.15,82,20,4.10,2020-07-19 01:37:06
563,Liaocheng,36.44,115.96,300.56,56,96,5.60,2020-07-19 01:37:07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#create hotel df
hotel_df = vacation_df.filter(["City Name", "Latitude", "Longitude"], axis=1)
hotel_df["Hotel Name"] = ""
hotel_df["Hotel City"] = ""
hotel_df["Hotel Country"] = ""
hotel_df


,City Name,Latitude,Longitude,Hotel Name,Hotel City,Hotel Country
0,Mataura,-46.19,168.86,,,
1,Jamestown,42.10,-79.24,,,
2,Lüderitz,-26.65,15.16,,,
3,Ushuaia,-54.80,-68.30,,,
4,Rikitea,-23.12,-134.97,,,
...,...,...,...,...,...,...
560,Vangaindrano,-23.35,47.60,,,
561,Jonesboro,35.84,-90.70,,,
562,Vila Franca do Campo,37.72,-25.43,,,
563,Liaocheng,36.44,115.96,,,


In [27]:
#set variables to capture details from query
name = []
city = []
country = []

#set up params

params = {"keyword": "hotel",
          "types": "hotel",
          "radius":5000, 
          "rankby":"distance",
          "key": g_key,}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#loop through coordinates to get responses
for index,row in vacation_df.iterrows():
    
    #get location from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    #add to params
    params["location"] = f"{latitude},{longitude}"
    
    #create URL for request
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    #extract results
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    #in case tehre are errors
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

        
    #print("------------")
    

NameError: name 'json' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
